# Autores iberoamericanos

In [1]:
import ast
import csv
import os
import pickle
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores8.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 2.08 s, sys: 25.8 ms, total: 2.11 s
Wall time: 2.1 s


In [3]:
autores_df.shape

(61857, 10)

In [4]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber,Berlin,Madrid,"(52.52000659999999, 13.404954)","(40.4167754, -3.7037902)"
1,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom,Helen Cooper,London,Madrid,"(51.5073509, -0.1277583)","(40.4167754, -3.7037902)"
2,Codrilo,2009,Spain,Roberto Aliaga Sánchez,Spain,Roberto Aliaga,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
3,"Félix, : el coleccionista de miedos",2009,Spain,Fina Casalderrey,Spain,Fina Casalderrey,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
4,En la laguna más profunda,2011,Colombia,Oscar Collazos,Colombia,Oscar Collazos,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"


## Autores iberoamericanos

In [5]:
paises_iberoamericanos = [
    # Español
    'Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Ecuador',
    'El Salvador', 'Guatemala', 'Honduras', 'México', 'Nicaragua', 'Panamá', 'Paraguay',
    'Perú', 'Puerto Rico', 'República Dominicana', 'Uruguay', 'Venezuela',
    # Inglés
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Ecuador',
    'El Salvador', 'Guatemala', 'Honduras', 'Mexico', 'Nicaragua', 'Panama', 'Paraguay',
    'Peru', 'Puerto Rico', 'Dominican Republic', 'Uruguay', 'Venezuela',
]

In [6]:
autores_iberoamericanos_df = autores_df[autores_df.PAIS_AUTOR.isin(paises_iberoamericanos)]

In [7]:
autores_iberoamericanos_df.shape

(13782, 10)

In [8]:
autores_iberoamericanos_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
4,En la laguna más profunda,2011,Colombia,Oscar Collazos,Colombia,Oscar Collazos,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
32,Galileo Galilei,2009,Spain,Guilherme de Almeida,Brazil,Guilherme de Almeida,Brasília,Madrid,"(-15.7942287, -47.8821658)","(40.4167754, -3.7037902)"
33,Haiku,2009,Argentina,Iris Rivera,Argentina,Iris Rivera,Buenos Aires,Buenos Aires,"(-34.6036844, -58.3815591)","(-34.6036844, -58.3815591)"
50,Sana que sana,2008,Spain,María Cristina Ramos,Argentina,María Cristina Ramos,Buenos Aires,Madrid,"(-34.6036844, -58.3815591)","(40.4167754, -3.7037902)"
54,Nicanor Parra : poemas ilustrados,2012,Chile,Nicanor Parra,Chile,Nicanor Parra,Santiago,Santiago,"(-33.4488897, -70.6692655)","(-33.4488897, -70.6692655)"


## Resultados

### Ciclos

In [9]:
ciclos_df = autores_iberoamericanos_df[
    (autores_iberoamericanos_df.PAIS_EJEMPLAR == autores_iberoamericanos_df.PAIS_AUTOR)
]

In [10]:
len(ciclos_df)

7054

#### Obras escritas por autores iberoamericanos y editadas en el mismo país de origen del autor

In [11]:
ciclos_df.TITULO.value_counts()

Obras completas                                                 28
Cuentos                                                         22
Antología                                                       21
María                                                           19
Antología poética                                               18
Obra poética                                                    15
El coronel no tiene quien le escriba                            14
Cuentos completos                                               14
Cien años de soledad                                            13
La vorágine                                                     13
Poesía                                                          11
Los grandes conflictos sociales y económicos de nuestra hist    10
Poesía completa                                                 10
Obras                                                           10
Teatro                                                        

#### Número de obras

In [12]:
ciclos_df.TITULO.nunique()

6043

#### Autores

In [13]:
ciclos_df.NOMBRE_LOCALIZADO.value_counts()

Gabriel García Márquez        204
Fernando Soto Aparicio         98
Jorge Luis Borges              98
Jairo Aníbal Niño              92
Javier Ocampo López            79
Eduardo Caballero Calderón     78
Octavio Paz                    76
José María Vargas Vila         76
Celso Román                    67
William Ospina                 57
Germán Espinosa                55
Germán Castro Caycedo          55
José Martí                     55
Manuel Mejía Vallejo           54
Tomás Carrasquilla             51
Andrés Caicedo                 48
Manuel Zapata Olivella         45
León de Greiff                 43
Jorge Isaacs                   43
Juan Manuel Roca               42
Ruth Rocha                     40
Alfredo Iriarte                39
Alfonso Reyes                  38
Oscar Collazos                 37
Roberto Fontanarrosa           36
Miguel Antonio Caro            36
Daniel Samper Pizano           34
José Emilio Pacheco            34
Piedad Bonnett                 30
Jorge Zalamea 

#### Número de autores

In [14]:
ciclos_df.NOMBRE_LOCALIZADO.nunique()

1306

#### Países

In [15]:
%%time

paises_cy = []
for i, x in ciclos_df.iterrows():
    pais_autor = x['PAIS_AUTOR']
    pais_ejemplar = x['PAIS_EJEMPLAR']
    paises_cy.append((pais_autor, pais_ejemplar))

CPU times: user 878 ms, sys: 3.7 ms, total: 882 ms
Wall time: 880 ms


In [16]:
paises_cy_c = Counter(paises_cy)

In [17]:
top_paises_cy = paises_cy_c.most_common()

In [18]:
top_paises_cy

[(('Colombia', 'Colombia'), 2883),
 (('Mexico', 'Mexico'), 1364),
 (('Argentina', 'Argentina'), 1346),
 (('Venezuela', 'Venezuela'), 280),
 (('Chile', 'Chile'), 247),
 (('Brazil', 'Brazil'), 239),
 (('Cuba', 'Cuba'), 223),
 (('Peru', 'Peru'), 199),
 (('Ecuador', 'Ecuador'), 99),
 (('Uruguay', 'Uruguay'), 79),
 (('Costa Rica', 'Costa Rica'), 25),
 (('Bolivia', 'Bolivia'), 20),
 (('El Salvador', 'El Salvador'), 13),
 (('Nicaragua', 'Nicaragua'), 10),
 (('Dominican Republic', 'Dominican Republic'), 10),
 (('Panama', 'Panama'), 7),
 (('Guatemala', 'Guatemala'), 5),
 (('Paraguay', 'Paraguay'), 4),
 (('Honduras', 'Honduras'), 1)]

### Rutas iberoamericanas

In [19]:
rutas_iberoamericanas_df = autores_iberoamericanos_df[
    (autores_iberoamericanos_df.PAIS_EJEMPLAR.isin(paises_iberoamericanos)) &
    (autores_iberoamericanos_df.PAIS_EJEMPLAR != autores_iberoamericanos_df.PAIS_AUTOR)
]

In [20]:
len(rutas_iberoamericanas_df)

3528

#### Obras escritas por autores iberoamericanos y editadas en Iberoamérica, pero en un país diferente del país de origen del autor

In [21]:
rutas_iberoamericanas_df.TITULO.value_counts()

Martín Fierro                                                   19
Antología poética                                               18
Cuentos                                                         12
La vorágine                                                     12
Antología                                                       11
Las venas abiertas de América Latina                            11
Poesía                                                          11
María                                                           10
Aura                                                             9
Antología personal                                               9
Cuentos completos                                                8
Obras completas                                                  8
El reino de este mundo                                           8
Obra selecta                                                     7
Poemas                                                        

#### Número de obras

In [22]:
rutas_iberoamericanas_df.TITULO.nunique()

2946

#### Autores

In [23]:
rutas_iberoamericanas_df.NOMBRE_LOCALIZADO.value_counts()

Pablo Neruda                 94
Carlos Fuentes               90
Mario Benedetti              78
Mario Vargas Llosa           58
Alejo Carpentier             57
Fernando Ayala               51
Horacio Quiroga              48
Gabriel García Márquez       48
Rubén Darío                  48
Eduardo Galeano              44
Marta Traba                  39
José Martí                   38
César Vallejo                37
Antonio Orlando Rodríguez    37
Jorge Luis Borges            35
Jorge Amado                  34
Darío Jaramillo              33
Juan Carlos Onetti           33
Alfonso Reyes                32
Paulo Freire                 32
José Hernández               30
Augusto Monterroso           29
Paulo Coelho                 28
José María Arguedas          28
Marta Harnecker              27
Rubem Fonseca                27
Isabel Allende               26
Gabriela Mistral             22
Ricardo Sánchez              21
Ernesto Cardenal             20
                             ..
Luciano 

#### Número de autores

In [24]:
rutas_iberoamericanas_df.NOMBRE_LOCALIZADO.nunique()

835

#### Países

In [25]:
%%time

paises_ri = []
for i, x in rutas_iberoamericanas_df.iterrows():
    pais_autor = x['PAIS_AUTOR']
    pais_ejemplar = x['PAIS_EJEMPLAR']
    paises_ri.append((pais_autor, pais_ejemplar))

CPU times: user 472 ms, sys: 0 ns, total: 472 ms
Wall time: 471 ms


In [26]:
paises_ri_c = Counter(paises_ri)

In [27]:
top_paises_ri = paises_ri_c.most_common()

In [28]:
top_paises_ri

[(('Argentina', 'Colombia'), 342),
 (('Argentina', 'Mexico'), 187),
 (('Mexico', 'Colombia'), 181),
 (('Peru', 'Colombia'), 150),
 (('Chile', 'Colombia'), 148),
 (('Cuba', 'Colombia'), 147),
 (('Uruguay', 'Argentina'), 147),
 (('Brazil', 'Colombia'), 128),
 (('Uruguay', 'Colombia'), 125),
 (('Cuba', 'Mexico'), 109),
 (('Brazil', 'Argentina'), 90),
 (('Colombia', 'Mexico'), 87),
 (('Uruguay', 'Mexico'), 87),
 (('Chile', 'Argentina'), 83),
 (('Peru', 'Argentina'), 81),
 (('Peru', 'Mexico'), 80),
 (('Panama', 'Mexico'), 73),
 (('Chile', 'Mexico'), 69),
 (('Venezuela', 'Colombia'), 67),
 (('Brazil', 'Mexico'), 59),
 (('Cuba', 'Argentina'), 59),
 (('Colombia', 'Argentina'), 49),
 (('Nicaragua', 'Colombia'), 39),
 (('Argentina', 'Venezuela'), 38),
 (('Mexico', 'Argentina'), 35),
 (('Cuba', 'Venezuela'), 35),
 (('Nicaragua', 'Mexico'), 33),
 (('Ecuador', 'Colombia'), 32),
 (('Venezuela', 'Mexico'), 28),
 (('Colombia', 'Venezuela'), 28),
 (('Uruguay', 'Venezuela'), 25),
 (('Honduras', 'Mexico'

### Rutas mundiales

In [29]:
rutas_mundiales_df = autores_iberoamericanos_df[
    ~autores_iberoamericanos_df.PAIS_EJEMPLAR.isin(paises_iberoamericanos)
]

In [30]:
len(rutas_mundiales_df)

3200

#### Obras escritas por autores iberoamericanos y editadas fuera de Iberoamérica

In [31]:
rutas_mundiales_df.TITULO.value_counts()

Antología poética                                               18
Obras completas                                                 15
Cuentos completos                                               11
La ciudad y los perros                                          10
Cuentos                                                          9
Poesía completa                                                  9
Poesía                                                           9
Martín Fierro                                                    8
La casa verde                                                    7
Conversación en la catedral                                      6
Cien años de soledad                                             6
Obra poética                                                     6
Juntacadáveres                                                   6
Antología                                                        6
Los pasos perdidos                                            

#### Número de obras

In [32]:
rutas_mundiales_df.TITULO.nunique()

2784

#### Autores

In [33]:
rutas_mundiales_df.NOMBRE_LOCALIZADO.value_counts()

Gabriel García Márquez       115
Mario Vargas Llosa            97
Jorge Luis Borges             83
Italo Calvino                 59
Pablo Neruda                  55
Octavio Paz                   54
Mario Benedetti               46
Alejandro Jodorowsky          44
Rubén Darío                   37
Alejo Carpentier              36
Carlos Fuentes                31
Guillermo Cabrera Infante     31
Adolfo Bioy Casares           30
Juan Carlos Onetti            30
Mario Bunge                   29
Cristina Peri Rossi           29
Isabel Allende                28
José Donoso                   27
Leonardo Boff                 27
José Martí                    27
Roberto Bolaño                24
Horacio Quiroga               23
Alfredo Bryce Echenique       22
Ramón Menéndez                20
Alberto Manguel               19
César Vallejo                 19
Jorge Amado                   19
Eduardo Galeano               19
Jorge Edwards                 19
Reinaldo Arenas               18
          

#### Número de autores

In [34]:
rutas_mundiales_df.NOMBRE_LOCALIZADO.nunique()

726

#### Países

In [35]:
%%time

paises_rw = []
for i, x in rutas_mundiales_df.iterrows():
    pais_autor = x['PAIS_AUTOR']
    pais_ejemplar = x['PAIS_EJEMPLAR']
    paises_rw.append((pais_autor, pais_ejemplar))

CPU times: user 412 ms, sys: 3.71 ms, total: 416 ms
Wall time: 415 ms


In [36]:
paises_rw_c = Counter(paises_rw)

In [37]:
top_paises_rw = paises_rw_c.most_common()

In [38]:
top_paises_rw

[(('Argentina', 'Spain'), 701),
 (('Cuba', 'Spain'), 387),
 (('Mexico', 'Spain'), 312),
 (('Peru', 'Spain'), 309),
 (('Chile', 'Spain'), 289),
 (('Colombia', 'Spain'), 224),
 (('Uruguay', 'Spain'), 214),
 (('Brazil', 'Spain'), 166),
 (('Nicaragua', 'Spain'), 78),
 (('Venezuela', 'Spain'), 60),
 (('Colombia', 'Francia'), 35),
 (('Panama', 'Spain'), 29),
 (('Colombia', 'United States'), 25),
 (('Honduras', 'Spain'), 21),
 (('Cuba', 'United States'), 19),
 (('Argentina', 'United States'), 19),
 (('Dominican Republic', 'Spain'), 17),
 (('Colombia', 'Germany'), 16),
 (('Colombia', 'Portugal'), 15),
 (('Guatemala', 'Spain'), 14),
 (('Ecuador', 'Spain'), 14),
 (('Bolivia', 'Spain'), 12),
 (('Peru', 'United States'), 12),
 (('Argentina', 'Francia'), 8),
 (('Brazil', 'Burma'), 8),
 (('Paraguay', 'Spain'), 8),
 (('Chile', 'China'), 8),
 (('Brazil', 'United States'), 7),
 (('Chile', 'United States'), 7),
 (('Colombia', 'New York (State)'), 7),
 (('Mexico', 'United States'), 7),
 (('El Salvador', 

## Grafos

In [39]:
%%time

coords = {}
for i, x in autores_df.iterrows():
    coords[x.PAIS_AUTOR] = x.GEOLOC_CAPITAL_AUTOR
    coords[x.PAIS_EJEMPLAR] = x.GEOLOC_CAPITAL_EJEMPLAR

CPU times: user 12.4 s, sys: 3.53 ms, total: 12.4 s
Wall time: 12.4 s


In [40]:
cycles_graph = nx.DiGraph()
r_iber_graph = nx.DiGraph()
r_wrld_graph = nx.DiGraph()

In [41]:
for k in paises_cy_c:
    if k[0] not in cycles_graph:
        lat, lng = coords[k[0]]
        cycles_graph.add_node(k[0], latitude=lat, longitude=lng, weight=paises_cy_c[k])
    
    cycles_graph.add_edge(k[0], k[0], weight=paises_cy_c[k])

In [42]:
for k in paises_ri_c:
    if k[0] not in r_iber_graph:
        lat, lng = coords[k[0]]
        r_iber_graph.add_node(k[0], latitude=lat, longitude=lng, weight=0)
    r_iber_graph.node[k[0]]['weight'] += paises_ri_c[k]
    
    if k[1] not in r_iber_graph:
        lat, lng = coords[k[1]]
        r_iber_graph.add_node(k[1], latitude=lat, longitude=lng, weight=0)
    r_iber_graph.node[k[1]]['weight'] += paises_ri_c[k]
    
    r_iber_graph.add_edge(k[0], k[1], weight=paises_ri_c[k])

In [43]:
for k in paises_rw_c:
    if k[0] not in r_wrld_graph:
        lat, lng = coords[k[0]]
        r_wrld_graph.add_node(k[0], latitude=lat, longitude=lng, weight=0)
    r_wrld_graph.node[k[0]]['weight'] += paises_rw_c[k]
    
    if k[1] not in r_wrld_graph:
        lat, lng = coords[k[1]]
        r_wrld_graph.add_node(k[1], latitude=lat, longitude=lng, weight=0)
    r_wrld_graph.node[k[1]]['weight'] += paises_rw_c[k]
    
    r_wrld_graph.add_edge(k[0], k[1], weight=paises_rw_c[k])

In [44]:
cycles_graph.number_of_nodes()

19

In [45]:
cycles_graph.number_of_edges()

19

In [46]:
r_iber_graph.number_of_nodes()

20

In [47]:
r_iber_graph.number_of_edges()

160

In [48]:
r_wrld_graph.number_of_nodes()

69

In [49]:
r_wrld_graph.number_of_edges()

123

## Filtering labels

In [50]:
def select_top_nodes(g, p):
    ns = [(n, d['weight']) for n, d in g.nodes_iter(data=True)]
    ns = sorted(ns, key=lambda x: x[1], reverse=True)
    amount = round(len(ns) * p)
    taken = ns[:amount]
    labels = [n for n, w in taken]
    return labels

In [51]:
for i in range(10, 100, 10):
    p = i / 100
    top_cy_labels = select_top_nodes(cycles_graph, p)
    for k in cycles_graph:
        if k in top_cy_labels:
            cycles_graph.node[k]['show_label_' + str(i)] = k
        else:
            cycles_graph.node[k]['show_label_' + str(i)] = ''

In [52]:
for i in range(10, 100, 10):
    p = i / 100
    top_ri_labels = select_top_nodes(r_iber_graph, p)
    for k in r_iber_graph:
        if k in top_ri_labels:
            r_iber_graph.node[k]['show_label_' + str(i)] = k
        else:
            r_iber_graph.node[k]['show_label_' + str(i)] = ''

In [53]:
for i in range(10, 100, 10):
    p = i / 100
    top_rw_labels = select_top_nodes(r_wrld_graph, p)
    for k in r_wrld_graph:
        if k in top_rw_labels:
            r_wrld_graph.node[k]['show_label_' + str(i)] = k
        else:
            r_wrld_graph.node[k]['show_label_' + str(i)] = ''

# Saving results

In [54]:
nx.write_gexf(cycles_graph, 'gephi/grafo_ciclos_iberoamericanos.gexf')
nx.write_gexf(r_iber_graph, 'gephi/grafo_rutas_iberoamericanas.gexf')
nx.write_gexf(r_wrld_graph, 'gephi/grafo_rutas_mundiales.gexf')

In [55]:
autores_iberoamericanos_df.to_csv('autores_iberoamericanos.csv', index=False)